In [1]:
!pip install bayesian-optimization

  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-py3-none-any.whl size=11685 sha256=bf4d5fcc49143d14e7c22d5ede47a39b8cb8de002f6ae0a481f7a1cdf277d48d
  Stored in directory: /root/.cache/pip/wheels/fd/9b/71/f127d694e02eb40bcf18c7ae9613b88a6be4470f57a8528c5b
Successfully built bayesian-optimization


In [2]:
!pip install catboost

     |████████████████████████████████| 69.2 MB 5.5 kB/s 


In [3]:
import numpy as np 
import matplotlib.pyplot as plt
import glob
import cv2
from sklearn.model_selection import train_test_split
import os
from keras.applications.densenet import DenseNet169, DenseNet121
from keras.applications.mobilenet import MobileNet
from sklearn import preprocessing
from google.colab import drive
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from bayes_opt import BayesianOptimization
from sklearn.metrics import mean_squared_error
from catboost import CatBoostClassifier
from sklearn.model_selection import KFold, cross_val_score
import lightgbm as lgb

## loading the dataset

In [4]:
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [6]:
SIZE = 224
images = []
images_labels = [] 

### importing each image and set the current directory name as its label 

In [7]:
ext = ['png', 'jpg', 'jpeg'] 

for directory_path in glob.glob("/content/gdrive/My Drive/seperated dataset/X-Ray Image DataSet/images/*"):
    splited = directory_path.split("/")
    label = splited[-1]
    print(label)
    for e in ext:
      for img_path in glob.glob(os.path.join(directory_path, "*." + e)):
        if label == 'Pneumonia':
          break
        else:
          #print(img_path)
           img = cv2.imread(img_path, cv2.IMREAD_COLOR)       
           img = cv2.resize(img, (SIZE, SIZE))
           images.append(img)
           images_labels.append(label)

Pneumonia
No_findings
Covid-19


In [8]:
images = np.array(images)
images_labels = np.array(images_labels)

In [9]:
images.shape

(625, 224, 224, 3)

In [10]:
np.unique(images_labels)

array(['Covid-19', 'No_findings'], dtype='<U11')

## Pre-processing

### label encoding

In [11]:
le = preprocessing.LabelEncoder()
le.fit(images_labels)
labels_encoded = le.transform(images_labels)

## Feature Extraction

### DenseNet169

In [12]:
dense_model = DenseNet169(include_top=False, input_shape=(SIZE, SIZE, 3), pooling='avg')

51888128/51877672 [==============================] - 1s 0us/step


In [13]:
dense_features = dense_model.predict(images)

In [14]:
features = dense_features.reshape(dense_features.shape[0], -1)

In [15]:
features.shape

(625, 1664)

### Mobilenet

In [16]:
model_mobile = MobileNet(weights='imagenet',include_top=False, input_shape=(SIZE, SIZE, 3), pooling='avg')

17235968/17225924 [==============================] - 0s 0us/step


In [17]:
mobile_features = model_mobile.predict(images)

In [18]:
features_2 = mobile_features.reshape(mobile_features.shape[0], -1)

In [19]:
features_2.shape

(625, 1024)

## Feature Combination

In [20]:
combined_features = np.hstack((dense_features, mobile_features))

## lightgbm classifier

In [30]:
params = {'learning_rate':0.2, 'n_iterations': 500, 'max_depth': 7, 'num_leaves': 105, 'n_estimators': 300, 'min_child_samples': 40 }


In [31]:
lgb_classifier = lgb.LGBMClassifier(**params)

## KFold cross validation

In [32]:
cv_r2_scores = cross_val_score(lgb_classifier, combined_features, labels_encoded, cv=5)

In [33]:
print(cv_r2_scores)

[0.952 0.992 0.976 0.976 0.968]


In [34]:
print("Mean 5-Fold R Squared: {}".format(np.mean(cv_r2_scores)))

Mean 5-Fold R Squared: 0.9728
